## 前処理目的
* 住民基本台帳と人口動態調査を用いた人口・死亡・出生における分析用データの作成

## 各種ステップ

### [STEP0 事前準備](#STEP0)
1. 必要なモジュールのimport
2. 市区町村マスタの読み込み
3. 政令指定都市マスタの読み込み

### [STEP1 住民基本台帳による人口データの前処理](#STEP1)
1. 指定フォルダにあるファイル情報の抽出（リスト、grob、ワイルドカード）
2. ファイルの読み込み（read_csv、type、assign、pd.concat）
3. 必要なカラムのみに制限
4. 文字列から数値への変換（dtype、連鎖代入、copy()、str.replace、astype）
5. 市区町村コードの追加（str[]）
6. 不要な行の除去①市区町村コードが欠損（dropna）
7. 不要な行の除去②都道府県集計を除去=末尾が"000"（!、条件抽出）
8. 不要な行の除去③政令指定都市を除去（リストを用いた条件抽出）
9. 最新市区町村情報の付与（pd.merge(how=***、on=***)）
10. 最新市区町村でのグループ集計（groupby().統計量）

### [STEP2 人口動態調査による人口データの前処理](#STEP2)
1. 指定フォルダにあるファイル情報の抽出
2. ファイルの一括読み込み
3. 文字列から数値への変換（余計な文字列データの除去（全角スペース\u3000））
4. 市区町村コードの作成
5. 集計を最小単位にする（str.isdecimal()）
6. 最新市区町村へのマージ
7. 最新市区町村での再集計

### [STEP3 人口に関する可視化向けファイルの作成](#STEP3)
1. 住民基本台帳と人口動態調査の前処理データの横結合
2. 不要な行を削除
3. STEP3-2のデータから政令指定都市のみを抽出
4. 政令指定都市で再集計
5. 政令指定都市以外の集計データとの縦結合
6. 出生率・死亡率の算出
7. カラム名の修正
8. データの並び替え政令指定都市以外の集計データとの縦結合
9. ファイルの出力


### STEP0 事前準備<a name='STEP0'>

In [1]:
#{コード4.1 事前準備}
#必要なライブラリの読み込み
import pandas as pd
import numpy as np
import os 
from glob import glob
import datetime

In [2]:
#[コード4.2 市区町村マスタの読み込み]
# 市区町村マスタの読み込み
j5_mst = pd.read_csv('./in/municipalitiy_mst.csv',encoding='cp932',
                     dtype = {'変換前市区町村コード':'object','最新都道府県コード':'object','最新市区町村コード':'object','政令指定都市フラグ':'object'}
                    )
print('j5_mst'); display(j5_mst.head())
print(len(j5_mst))


j5_mst


,変換前市区町村コード,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,政令指定都市フラグ
0,01100,01,北海道,01100,札幌市,1
1,01101,01,北海道,01101,札幌市中央区,NaN
2,01102,01,北海道,01102,札幌市北区,NaN
3,01103,01,北海道,01103,札幌市東区,NaN
4,01104,01,北海道,01104,札幌市白石区,NaN


425


In [3]:
# {コード4.3 政令指定都市マスタの読み込み}
# 政令指定都市マスタの読み込み
seirei_mst = pd.read_csv('./in/seireicity_mst.csv',encoding='cp932',
                     dtype = {'変換前市区町村コード':'object','政令指定都市コード':'object'}
                    )
print('seirei_mst'); display(seirei_mst.head())
print(len(seirei_mst))

seirei_mst


,変換前市区町村コード,政令指定都市コード,政令指定都市名
0,01101,01100,札幌市
1,01102,01100,札幌市
2,01103,01100,札幌市
3,01104,01100,札幌市
4,01105,01100,札幌市


38


### STEP1 住民基本台帳による人口データの前処理<a name='STEP1'>

In [4]:
# {コード4.4 指定フォルダにあるファイル情報の抽出}
current_dir = './in/juuminkihon'

#外国人が対象になっているかどうかでカラムが変化することから、２００９～２０１２と2013～2018でデータを分ける
all_csv_files1_1 = [file
                      for file in glob(os.path.join(current_dir, '*ssjin*.xls'))]
all_csv_files1_2 = [file
                      for file in glob(os.path.join(current_dir, '*nsjin*.xls'))]

In [5]:
all_csv_files1_1

['./in/juuminkihon/0903ssjin.xls',
 './in/juuminkihon/1003ssjin.xls',
 './in/juuminkihon/1203ssjin.xls',
 './in/juuminkihon/1103ssjin.xls']

In [6]:
all_csv_files1_2

['./in/juuminkihon/1607nsjin.xls',
 './in/juuminkihon/1307nsjin.xls',
 './in/juuminkihon/1407nsjin.xls',
 './in/juuminkihon/1707nsjin.xls',
 './in/juuminkihon/1807nsjin.xls',
 './in/juuminkihon/1507nsjin.xls']

In [7]:
# {コード4.5 ファイルの読み込み1}
lists1=[]
for file in all_csv_files1_1:
    df1_1 = pd.read_excel(file, index_col=None, header=None, skiprows=4, 
                        names=('団体コード','都道府県名','市区町村名','人口_男','人口_女','人口_計','世帯数計','転入者数_計','出生者数','住民票記載数_その他_計','住民票記載数計','転出者数_計','死亡者数','住民票消除数_その他_計','住民票消除数計','住民票増減数','住民票増加率','自然増加数','自然増加率','社会増加数','社会増加率'
                               ),
                        dtype = {'団体コード':'object'}
                        ).assign(集計年 = "20"+file[-13:-11])
    lists1.append(df1_1)
for file in all_csv_files1_2:
    df1_2 = pd.read_excel(file, index_col=None, header=None, skiprows=4, 
                        names=('団体コード','都道府県名','市区町村名','人口_男','人口_女','人口_計','世帯数_日本人住民','世帯数_複数国籍','世帯数計','転入者数_国内','転入者数_国外','転入者数_計','出生者数','住民票記載数_その他_帰化等','住民票記載数_その他_その他','住民票記載数_その他_計','住民票記載数計','転出者数_国内','転出者数_国外','転出者数_計','死亡者数','住民票消除数_その他_国籍喪失','住民票消除数_その他_その他','住民票消除数_その他_計','住民票消除数計','住民票増減数','住民票増加率','自然増加数','自然増加率','社会増加数','社会増加率'
                               ),
                        dtype = {'団体コード':'object'}
                        ).assign(集計年 = "20"+file[-13:-11])
    lists1.append(df1_2)
df1_base1 = pd.concat(lists1, axis=0, sort=False)

print('df1_base1'); display(df1_base1.head())
print(len(df1_base1))

df1_base1


,団体コード,都道府県名,市区町村名,人口_男,人口_女,人口_計,世帯数計,転入者数_計,出生者数,住民票記載数_その他_計,...,世帯数_日本人住民,世帯数_複数国籍,転入者数_国内,転入者数_国外,住民票記載数_その他_帰化等,住民票記載数_その他_その他,転出者数_国内,転出者数_国外,住民票消除数_その他_国籍喪失,住民票消除数_その他_その他
0,NaN,合計,NaN,62105515,64970668,127076183,52877802,5510512,1088488,88772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,010006,北海道,NaN,2631725,2911831,5543556,2637145,272970,40936,2390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,011002,北海道,札幌市,888900,996039,1884939,944872,127901,14739,416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,011011,北海道,札幌市中央区,93434,112818,206252,118974,21839,1617,72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,011029,北海道,札幌市北区,130642,142935,273577,134723,16623,2107,67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


3765


In [8]:
#{コード4.6 必要なカラムのみに制限}

df1_base2 = df1_base1[['集計年','団体コード','人口_計']]
df1_base2

,集計年,団体コード,人口_計
0,2009,NaN,127076183
1,2009,010006,5543556
2,2009,011002,1884939
3,2009,011011,206252
4,2009,011029,273577
5,2009,011037,252688
6,2009,011045,203579
7,2009,011053,208476
8,2009,011061,147397
9,2009,011070,209883


In [9]:
# {コード4.7 市区町村コードの追加}
df1_base3=df1_base2.assign(市区町村コード=df1_base2['団体コード'].str[0:5])

df1_base3.head(3)

,集計年,団体コード,人口_計,市区町村コード
0,2009,NaN,127076183,NaN
1,2009,010006,5543556,01000
2,2009,011002,1884939,01100


In [10]:
# {コード4.8 NaN のレコードの除去1}
df1_base4 = df1_base3.dropna(subset=['市区町村コード']) 
df1_base4.head(3)

,集計年,団体コード,人口_計,市区町村コード
1,2009,010006,5543556,01000
2,2009,011002,1884939,01100
3,2009,011011,206252,01101


In [11]:
# {コード4.9 末尾が000 のレコードの除去2}
df1_base5=df1_base4[df1_base4['団体コード'].str[2:5] != "000"]

df1_base5.head(3)

,集計年,団体コード,人口_計,市区町村コード
2,2009,011002,1884939,01100
3,2009,011011,206252,01101
4,2009,011029,273577,01102


In [12]:
# {コード4.10 政令指定都市の除去}
j5_seirei = j5_mst.最新市区町村コード[(j5_mst['政令指定都市フラグ'] == '1')]
           
print(j5_seirei.head(3))

df1_base6 = df1_base5

for seirei in j5_seirei:
    df1_base6 = df1_base6[(df1_base6['市区町村コード'] != seirei)]

df1_base6.head(3)

0      01100
300    14100
319    14130
Name: 最新市区町村コード, dtype: object


,集計年,団体コード,人口_計,市区町村コード
3,2009,011011,206252,01101
4,2009,011029,273577,01102
5,2009,011037,252688,01103


In [13]:
#{コード4.11 最新市区町村情報の付与}

df1_base7 = pd.merge(df1_base6, j5_mst, left_on='市区町村コード', right_on='変換前市区町村コード', how='inner')
df1_base7

,集計年,団体コード,人口_計,市区町村コード,変換前市区町村コード,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,政令指定都市フラグ
0,2009,011011,206252,01101,01101,01,北海道,01101,札幌市中央区,NaN
1,2010,011011,209934,01101,01101,01,北海道,01101,札幌市中央区,NaN
2,2012,011011,216307,01101,01101,01,北海道,01101,札幌市中央区,NaN
3,2011,011011,212666,01101,01101,01,北海道,01101,札幌市中央区,NaN
4,2016,011011,228172,01101,01101,01,北海道,01101,札幌市中央区,NaN
5,2013,011011,219066,01101,01101,01,北海道,01101,札幌市中央区,NaN
6,2014,011011,222392,01101,01101,01,北海道,01101,札幌市中央区,NaN
7,2017,011011,229800,01101,01101,01,北海道,01101,札幌市中央区,NaN
8,2018,011011,231394,01101,01101,01,北海道,01101,札幌市中央区,NaN
9,2015,011011,224928,01101,01101,01,北海道,01101,札幌市中央区,NaN


In [14]:
# {コード4.12 merge 実施時の確認例}
df1_che1 = pd.merge(df1_base6, j5_mst, left_on='市区町村コード', right_on='変換前市区町村コード', how='left')
df1_che2 = df1_che1[df1_che1["最新市区町村コード"].isnull()]
df1_che3 = df1_che2[(df1_che2['市区町村コード'].str[3:4] != "3") & (df1_che2['市区町村コード'].str[4:5] != "0")]

In [15]:
# {コード4.13 最新市区町村でのグループ集計}
df1_base8 = df1_base7.groupby(['集計年','最新都道府県コード','最新都道府県名','最新市区町村コード','最新市区町村名'],
                              as_index=False
                             ).sum()
print('df1_base8'); display(df1_base8.head())
print(len(df1_base8))

df1_base8


,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計
0,2009,01,北海道,01101,札幌市中央区,206252
1,2009,01,北海道,01102,札幌市北区,273577
2,2009,01,北海道,01103,札幌市東区,252688
3,2009,01,北海道,01104,札幌市白石区,203579
4,2009,01,北海道,01105,札幌市豊平区,208476


3486


### STEP2 人口動態調査による人口データの前処理<a name='STEP2'>

In [16]:
#{コード4.14 指定フォルダにあるファイル情報の抽出}
current_dir = './in/jinkodoutai'

all_csv_files2 = [file for current_dir, subdir, files in os.walk(current_dir)
                     for file in glob(os.path.join(current_dir, '*.csv'))]

all_csv_files2

['./in/jinkodoutai/2013/a020047.csv',
 './in/jinkodoutai/2013/a020013.csv',
 './in/jinkodoutai/2013/a020001.csv',
 './in/jinkodoutai/2013/a020014.csv',
 './in/jinkodoutai/2014/a020047.csv',
 './in/jinkodoutai/2014/a020013.csv',
 './in/jinkodoutai/2014/a020001.csv',
 './in/jinkodoutai/2014/a020014.csv',
 './in/jinkodoutai/2015/a020047.csv',
 './in/jinkodoutai/2015/a020013.csv',
 './in/jinkodoutai/2015/a020001.csv',
 './in/jinkodoutai/2015/a020014.csv',
 './in/jinkodoutai/2012/a020047.csv',
 './in/jinkodoutai/2012/a020013.csv',
 './in/jinkodoutai/2012/a020001.csv',
 './in/jinkodoutai/2012/a020014.csv',
 './in/jinkodoutai/2009/a020047.csv',
 './in/jinkodoutai/2009/a020013.csv',
 './in/jinkodoutai/2009/a020001.csv',
 './in/jinkodoutai/2009/a020014.csv',
 './in/jinkodoutai/2017/a020047.csv',
 './in/jinkodoutai/2017/a020013.csv',
 './in/jinkodoutai/2017/a020001.csv',
 './in/jinkodoutai/2017/a020014.csv',
 './in/jinkodoutai/2010/a020047.csv',
 './in/jinkodoutai/2010/a020013.csv',
 './in/jinko

In [17]:
# {コード4.15 ファイルの一括読み込み}
lists2=[]
for files2 in all_csv_files2:
    df2 = pd.read_csv(files2, skiprows=5, encoding='cp932',
                      names=('地域情報','出生数','出生数_2500g未満','死亡数','死亡数_乳児','死亡数_新生児','死産数_自然死産','死産数_人工死産','周産期死亡_総数',
                            '周産期死亡_22週以後の死産数','周産期死亡_早期新生児死亡数','婚姻件数','離婚件数'
                            ),
                    ).assign(集計年 = "20"+files2[-14:-12])
    lists2.append(df2)
df2_base1 = pd.concat(lists2, axis=0, sort=False)

df2_base1.head(3)

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,周産期死亡_早期新生児死亡数,婚姻件数,離婚件数,集計年
0,４７沖 縄,17209,1957,10956,30,23,219,224,77,66,11,8803,3651,2013
1,市 部,13439,1508,8266,23,17,171,180,59,50,9,6812,2887,2013
2,郡 部,3770,449,2690,7,6,48,44,18,16,2,1991,764,2013


In [18]:
# {コード4.16 文字列から数値への変換}
#スキル：余計な文字列データの除去（全角スペース\u3000）
df2_base2 = df2_base1.copy()

df2_base2['出生数_int'] = df2_base2['出生数'].replace('-', '0').replace('　　　', '0').fillna(0.0).astype(np.int64)
df2_base2['死亡数_int'] = df2_base2['死亡数'].replace('-', '0').replace('　　　', '0').fillna(0.0).astype(np.int64)

print(df2_base2['出生数_int'].dtype)
print(df2_base2['死亡数_int'].dtype)

int64
int64


In [19]:
# {コード4.17 市区町村コードの作成}
df2_base3=df2_base2.assign(市区町村コード=df2_base2['地域情報'].str[0:5])

df2_base3.head(3)

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,周産期死亡_早期新生児死亡数,婚姻件数,離婚件数,集計年,出生数_int,死亡数_int,市区町村コード
0,４７沖 縄,17209,1957,10956,30,23,219,224,77,66,11,8803,3651,2013,17209,10956,４７沖 縄
1,市 部,13439,1508,8266,23,17,171,180,59,50,9,6812,2887,2013,13439,8266,市 部
2,郡 部,3770,449,2690,7,6,48,44,18,16,2,1991,764,2013,3770,2690,郡 部


In [20]:
# {コード4.18 レコードを市区町村のみにする}
#数字のみのレコードにするメソッド：str.isdecimal()

df2_base4 = df2_base3[df2_base3['市区町村コード'].str.isdecimal() == True]

df2_base4.head(3)

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,周産期死亡_早期新生児死亡数,婚姻件数,離婚件数,集計年,出生数_int,死亡数_int,市区町村コード
10,47201那覇市,3495,350,2492,4,2,49,55,18,16,2,1902,789,2013,3495,2492,47201
11,47205宜野湾市,1282,153,547,5,3,15,13,7,6,1,683,276,2013,1282,547,47205
12,47207石垣市,666,63,402,1,1,9,10,2,1,1,334,173,2013,666,402,47207


In [21]:
#{コード4.19 最新市区町村へのマージ}
df2_base5 = pd.merge(df2_base4, j5_mst, left_on='市区町村コード', right_on='変換前市区町村コード', how='inner')

df2_base5.head(3)

,地域情報,出生数,出生数_2500g未満,死亡数,死亡数_乳児,死亡数_新生児,死産数_自然死産,死産数_人工死産,周産期死亡_総数,周産期死亡_22週以後の死産数,...,集計年,出生数_int,死亡数_int,市区町村コード,変換前市区町村コード,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,政令指定都市フラグ
0,47201那覇市,3495,350,2492,4,2,49,55,18,16,...,2013,3495,2492,47201,47201,47,沖縄県,47201,那覇市,NaN
1,47201那覇市,3306,360,2520,10,2,36,58,8,7,...,2014,3306,2520,47201,47201,47,沖縄県,47201,那覇市,NaN
2,47201那覇市,3296,321,2633,6,2,51,33,13,12,...,2015,3296,2633,47201,47201,47,沖縄県,47201,那覇市,NaN


In [22]:
#{コード4.20 最新市区町村での再集計}

df2_base6 = df2_base5.groupby(['集計年','最新都道府県コード','最新都道府県名','最新市区町村コード','最新市区町村名'], as_index=False).sum()

df2_base6.head(3)

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,出生数_int,死亡数_int
0,2009,01,北海道,01101,札幌市中央区,1580,1571
1,2009,01,北海道,01102,札幌市北区,2052,2057
2,2009,01,北海道,01103,札幌市東区,2221,1860


### STEP3 人口に関するデータ分析向けファイルの作成<a name='STEP3'>

In [23]:
# {コード4.21 住民基本台帳と人口動態調査の前処理データの横結合}
df3_base1 = pd.merge(df1_base8, df2_base6, on=['集計年','最新都道府県コード','最新都道府県名','最新市区町村コード','最新市区町村名'] ,
                     how='outer'
                    )
print(df3_base1)

#差分の調査
df3_che1 = df3_base1[df3_base1["人口_計"].isnull()]
df3_che2 = df3_base1[df3_base1["出生数_int"].isnull()]

print(df3_che1)
print(df3_che2)

       集計年 最新都道府県コード 最新都道府県名 最新市区町村コード  最新市区町村名      人口_計  出生数_int  死亡数_int
0     2009        01     北海道     01101   札幌市中央区  206252.0     1580     1571
1     2009        01     北海道     01102    札幌市北区  273577.0     2052     2057
2     2009        01     北海道     01103    札幌市東区  252688.0     2221     1860
3     2009        01     北海道     01104   札幌市白石区  203579.0     1831     1581
4     2009        01     北海道     01105   札幌市豊平区  208476.0     1721     1540
5     2009        01     北海道     01106    札幌市南区  147397.0      871     1362
6     2009        01     北海道     01107    札幌市西区  209883.0     1685     1654
7     2009        01     北海道     01108   札幌市厚別区  129604.0      819      934
8     2009        01     北海道     01109   札幌市手稲区  138794.0      953     1091
9     2009        01     北海道     01110   札幌市清田区  114689.0      773      856
10    2009        01     北海道     01202      函館市  284910.0     1889     3322
11    2009        01     北海道     01203      小樽市  135500.0      729     1752
12    2009  

In [24]:
# {コード4.22 不要な行を削除}
# 相模原市を除去していい理由＝2010年4月に政令指定都市となり、1月・4月締めでデータの集計期間が異なるため
df3_base2 = df3_base1.dropna(subset=['人口_計']) 

df3_base2.head(3)

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数_int,死亡数_int
0,2009,01,北海道,01101,札幌市中央区,206252.0,1580,1571
1,2009,01,北海道,01102,札幌市北区,273577.0,2052,2057
2,2009,01,北海道,01103,札幌市東区,252688.0,2221,1860


In [25]:
# {コード4.23 政令指定都市で再集計}
df3_sei1 = pd.merge(df3_base2, seirei_mst, left_on='最新市区町村コード', right_on='変換前市区町村コード', how='inner')
df3_sei1.head(3)

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数_int,死亡数_int,変換前市区町村コード,政令指定都市コード,政令指定都市名
0,2009,01,北海道,01101,札幌市中央区,206252.0,1580,1571,01101,01100,札幌市
1,2010,01,北海道,01101,札幌市中央区,209934.0,1751,1679,01101,01100,札幌市
2,2011,01,北海道,01101,札幌市中央区,212666.0,1706,1710,01101,01100,札幌市


In [26]:
# {コード4.24 政令指定都市で再集計}
df3_sei2 = df3_sei1.groupby(['集計年','最新都道府県コード','最新都道府県名','政令指定都市コード','政令指定都市名'], as_index=False).sum()

df3_sei2.head(3)

,集計年,最新都道府県コード,最新都道府県名,政令指定都市コード,政令指定都市名,人口_計,出生数_int,死亡数_int
0,2009,01,北海道,01100,札幌市,1884939.0,14506,14506
1,2009,14,神奈川県,14100,横浜市,3605951.0,32111,25544
2,2009,14,神奈川県,14130,川崎市,1359063.0,14276,8783


In [27]:
# {コード4.25 結合データとのカラム名統一}
df3_sei3 = df3_sei2.rename(columns={  "政令指定都市コード":"最新市区町村コード",
                                      "政令指定都市名":"最新市区町村名",
                                    }
                            )
df3_sei3.head(3)

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数_int,死亡数_int
0,2009,01,北海道,01100,札幌市,1884939.0,14506,14506
1,2009,14,神奈川県,14100,横浜市,3605951.0,32111,25544
2,2009,14,神奈川県,14130,川崎市,1359063.0,14276,8783


In [28]:
# {コード4.26 政令指定都市データの縦結合}
df3_base3 = pd.concat([df3_base2,df3_sei3],sort=False)

df3_base3

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数_int,死亡数_int
0,2009,01,北海道,01101,札幌市中央区,206252.0,1580,1571
1,2009,01,北海道,01102,札幌市北区,273577.0,2052,2057
2,2009,01,北海道,01103,札幌市東区,252688.0,2221,1860
3,2009,01,北海道,01104,札幌市白石区,203579.0,1831,1581
4,2009,01,北海道,01105,札幌市豊平区,208476.0,1721,1540
5,2009,01,北海道,01106,札幌市南区,147397.0,871,1362
6,2009,01,北海道,01107,札幌市西区,209883.0,1685,1654
7,2009,01,北海道,01108,札幌市厚別区,129604.0,819,934
8,2009,01,北海道,01109,札幌市手稲区,138794.0,953,1091
9,2009,01,北海道,01110,札幌市清田区,114689.0,773,856


In [29]:
# {コード4.27 出生率・死亡率を算出}
#スキル：四則演算、round()
df3_base4 = df3_base3.copy()

df3_base4['出生率'] = (df3_base4['出生数_int'] / df3_base4['人口_計'] * 100).round(3)
df3_base4['死亡率'] = (df3_base4['死亡数_int'] / df3_base4['人口_計'] * 100).round(3)

df3_base4

,集計年,最新都道府県コード,最新都道府県名,最新市区町村コード,最新市区町村名,人口_計,出生数_int,死亡数_int,出生率,死亡率
0,2009,01,北海道,01101,札幌市中央区,206252.0,1580,1571,0.766,0.762
1,2009,01,北海道,01102,札幌市北区,273577.0,2052,2057,0.750,0.752
2,2009,01,北海道,01103,札幌市東区,252688.0,2221,1860,0.879,0.736
3,2009,01,北海道,01104,札幌市白石区,203579.0,1831,1581,0.899,0.777
4,2009,01,北海道,01105,札幌市豊平区,208476.0,1721,1540,0.826,0.739
5,2009,01,北海道,01106,札幌市南区,147397.0,871,1362,0.591,0.924
6,2009,01,北海道,01107,札幌市西区,209883.0,1685,1654,0.803,0.788
7,2009,01,北海道,01108,札幌市厚別区,129604.0,819,934,0.632,0.721
8,2009,01,北海道,01109,札幌市手稲区,138794.0,953,1091,0.687,0.786
9,2009,01,北海道,01110,札幌市清田区,114689.0,773,856,0.674,0.746


In [30]:
# {コード4.28 列名の修正}
df3_base5 = df3_base4.rename(
                        columns={"最新都道府県コード":"都道府県コード",
                                 "最新都道府県名":"都道府県名",
                                 "最新市区町村コード":"市区町村コード",
                                 "最新市区町村名":"市区町村名",
                                 "人口_計":"人口数（人）",
                                 "出生数_int":"出生数（人）",
                                 "死亡数_int":"死亡数（人）",
                                 "出生率":"出生率（％）",
                                 "死亡率":"死亡率（％）"
                                }
                            )
df3_base5.head(3)

,集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,人口数（人）,出生数（人）,死亡数（人）,出生率（％）,死亡率（％）
0,2009,01,北海道,01101,札幌市中央区,206252.0,1580,1571,0.766,0.762
1,2009,01,北海道,01102,札幌市北区,273577.0,2052,2057,0.750,0.752
2,2009,01,北海道,01103,札幌市東区,252688.0,2221,1860,0.879,0.736


In [31]:
# {コード4.29 データの並び替え}
df3_out = df3_base5.sort_values(['集計年','市区町村コード'])

print('df3_out'); display(df3_out.head())
print(len(df3_out))

df3_out


,集計年,都道府県コード,都道府県名,市区町村コード,市区町村名,人口数（人）,出生数（人）,死亡数（人）,出生率（％）,死亡率（％）
0,2009,01,北海道,01100,札幌市,1884939.0,14506,14506,0.770,0.770
0,2009,01,北海道,01101,札幌市中央区,206252.0,1580,1571,0.766,0.762
1,2009,01,北海道,01102,札幌市北区,273577.0,2052,2057,0.750,0.752
2,2009,01,北海道,01103,札幌市東区,252688.0,2221,1860,0.879,0.736
3,2009,01,北海道,01104,札幌市白石区,203579.0,1831,1581,0.899,0.777


3524


In [32]:
# {コード4.30 ファイルの出力}
now = datetime.datetime.now()
df3_out.to_csv('./out/ratio_'+ now.strftime('%Y%m%d') +'_name.csv',
                header=True, index=False, float_format = '%.3f')